In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import cPickle as pickle
import gefry2

import scipy.stats as st

Load an existing problem file

In [3]:
with open('./problem.pkl') as f:
    P, obs, sigma = pickle.load(f)

# Assigning distributions

First, we need to pick some probability distributions, one for each detector. These can be any of the distributions in [`scipy.stats`](http://docs.scipy.org/doc/scipy/reference/stats.html). Actually, it just has to implement the `.rvs()` method and if you need something custom I can show you what to do.

**NOTE**: be careful about the spread, as there are not any checks to that count rates are positive and weird things might happen. E.g. if you picked a normal error distribution with variance of 1000 and add it to a base response of 10, you're probably going to get a lot of negative count rates and it'll get weird. 

**NOTE**: I'm assuming all the error distributions are independent. If you need correlated error distributions, I can implement that.

As an example, I'm going to make all the detector responses add a random sample from a [triangular distribution](http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.triang.html#scipy.stats.triang) and I'm going to use the same error distribution for all detectors. 

In [4]:
error_dists = [st.triang(1)] * len(P.detectors) # Make 15 copies of st.triang, one for each detector

First, you must construct an object representing the distributions. These are pretty flexible and I implemented exactly what you asked (that is, just specifying a distribution for each detector) for as `background_terms.PartialRandomField`. It has a convenience method that constructs the proper object given a list of detectors and matching list of distributions. Use it like this:

In [5]:
background_dist = gefry2.background_terms.PartialRandomField.from_detectors(P.detectors, error_dists)

In [6]:
background_dist(P.detectors[0]).mean()

0.66666666666666663

It is possible to construct a problem with the detector error distributions already included (by passing the appropriate distribution object when building it), but I'm assuming you're going to be using the pre-made problem that I have already generated, which does not have distributions.

I have included a convenience function that takes an old problem specification and upgrades it to the new version. The new version will have all the same parameters as the old one, but it will also upgrade the internals to the newest version and add on the distributions if you provide them. Just pass your new background term and it will replace the old one.

In [7]:
new_P = gefry2.Problem.upgrade_problem(P, background=background_dist)

`new_P` is just like `P` was in terms of parameters, but the internals have been upgraded and also the error distributions tacked on. For example,

In [8]:
new_P.domain == P.domain ## These are the same

True

In [9]:
# These are not the same, because we added on the distributions
new_P.detectors == P.detectors

True

In [10]:
# But the other parameters are the same for the detectors, e.g. the location:
all([l.loc == r.loc for l, r in zip(new_P.detectors, P.detectors)])

True

Note that you **cannot** resave this upgraded object to reuse. This is a fundamental limitation of Python pickles; you just need to call the upgrade every time or generate a real problem file with the background term included. E.g., this won't work:

In [11]:
with open('upgraded_problem.pkl', 'w') as f:
    pickle.dump([new_P, obs, sigma], f)

TypeError: can't pickle instancemethod objects

# Getting the detector measurements

Everything works like it used to, only you have a new option to control background terms in the `__call__` method.

In [12]:
# The methods expect to be called with a `Source` object, so I'm going to make one here that I will use as an example.
test_src = gefry2.Source([20, 30], 1e9)

## Get detector **reference** response (i.e., using the *actual* source location) without error and without background

In [13]:
new_P(src_hyp=None, uncertain=False, background=False) # Constant (not random)

array([  59.,  169.,    1.,  305.,  176.,   35.,   86.,  346.,  310.,  211.])

## Get detector reference response with nominal [mean] value of background

In [14]:
new_P(src_hyp=None, uncertain=False, background=True) # Also constant

array([  60.,  170.,    2.,  306.,  177.,   36.,   87.,  347.,  311.,  212.])

## Get detector response to arbitrary source, without Poisson counting errors and without background

In [15]:
new_P(src_hyp=test_src, uncertain=False, background=False) # Constant

array([  7.80000000e+02,   4.90000000e+01,   2.62100000e+03,
         3.00000000e+00,   7.00000000e+01,   3.00000000e+00,
         2.00000000e+00,   1.00000000e+00,   4.00000000e+00,
         5.00000000e+00])

## Get detector response to arbitrary source, with Poisson counting errors and without background

In [16]:
new_P(src_hyp=test_src, uncertain=True, background=False) # This is sampled randomly

array([  7.97000000e+02,   4.50000000e+01,   2.61200000e+03,
         2.00000000e+00,   7.50000000e+01,   1.00000000e+00,
         1.00000000e+00,   2.00000000e+00,   2.00000000e+00,
         5.00000000e+00])

## Get detector response to arbitrary source, without Poisson counting errors and with *nominal* background

In [17]:
new_P(src_hyp=test_src, uncertain=False, background=True) # Constant

array([  7.81000000e+02,   5.00000000e+01,   2.62200000e+03,
         4.00000000e+00,   7.10000000e+01,   4.00000000e+00,
         3.00000000e+00,   2.00000000e+00,   5.00000000e+00,
         6.00000000e+00])

## Get detector response to arbitrary source, with Poisson counting errors and with randomly sampled background

In [18]:
new_P(src_hyp=test_src, uncertain=True, background=True) # Random

array([  7.38000000e+02,   4.40000000e+01,   2.60400000e+03,
         1.00000000e+00,   7.30000000e+01,   4.00000000e+00,
         4.00000000e+00,   2.00000000e+00,   4.00000000e+00,
         4.00000000e+00])

Note one combination is missing, which is "without Poisson errors and with randomly sampled background". I can add it if you need.